# Problem description

This notebook uses CP Optimizer to solve Sudoku problems and an optimization variant.

<img align="left" width="500" src="./the_telegraph.png">

# Reading the data

This is the grid of the so-called "world's hardest sudoku" by Arto Inkala, 2012.

In [1]:
WorldsHardestSudoku = """
                8 . . | . . . | . . . 
                . . 3 | 6 . . | . . .   
                . 7 . | . 9 . | 2 . .   
                ------+-------+------
                . 5 . | . . 7 | . . .   
                . . . | . 4 5 | 7 . .   
                . . . | 1 . . | . 3 .   
                ------+-------+------
                . . 1 | . . . | . 6 8   
                . . 8 | 5 . . | . 1 .  
                . 9 . | . . . | 4 . .  
"""

def Read(Grid,size=9):
    chars = [0 if c == '.' else int(c) for c in Grid if c in '.0123456789']
    return [chars[i*size: (i+1)*size] for i in range(0,size)]

G = Read(WorldsHardestSudoku)
print(G)

[[8, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 3, 6, 0, 0, 0, 0, 0], [0, 7, 0, 0, 9, 0, 2, 0, 0], [0, 5, 0, 0, 0, 7, 0, 0, 0], [0, 0, 0, 0, 4, 5, 7, 0, 0], [0, 0, 0, 1, 0, 0, 0, 3, 0], [0, 0, 1, 0, 0, 0, 0, 6, 8], [0, 0, 8, 5, 0, 0, 0, 1, 0], [0, 9, 0, 0, 0, 0, 4, 0, 0]]


# Modeling the problem with CP Optimizer

In [2]:
# Import CP Optimizer modelization functions
from docplex.cp.model import *

# Create model object
model = CpoModel()

# Decision variables: x[r][c] is the value at row r column c
x = [ [integer_var(1,9) for c in range(9)] for r in range(9) ]

# Constraints: input grid values
model.add([x[r][c]==G[r][c] for r in range(9) for c in range(9) if G[r][c]!=0 ])            
            
# Constraints: different values in each row
model.add([all_diff([x[r][c] for c in range(9)]) for r in range(9)])

# Constraints: different values in each column    
model.add([all_diff([x[r][c] for r in range(9)]) for c in range(9)  ])

# Constraints: different values in each sub-square 
model.add([all_diff([x[r][c] for r in range(3*sr,3*sr+3) for c in range(3*sc,3*sc+3)]) for sr in range(3) for sc in range(3) ])

# Solving the problem with CP Optimizer automatic search

The model can be solved by calling CP Optimizer's automatic search:

In [3]:
# Solve the model
sol = model.solve(trace_log=True, LogPeriod=1000000)

 ! ----------------------------------------------------------------------------
 ! Satisfiability problem - 81 variables, 48 constraints
 ! LogPeriod            = 1000000
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 122.9 (before), 122.9 (after)
 !  . Memory usage      : 335.2 kB (before), 335.2 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                   1158  0.02s        3         5 != _INT_32
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 17591
 ! Number of fails        : 8670
 ! Total memory usage     : 5.2 MB (5.1 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.03s (0.02s engine + 0.

# Displaying the input grid and the solution

In [4]:
print("Input grid:")
for r in range(9):
    print('                      ', end='')
    for c in range(9):
        if G[r][c]==0:
            s = '.'
            s = '\x1b[1m'+s
        else:
            s = str(G[r][c])
            s = '\x1b[1;01m'+s  
        print(s+'\x1b[0m'+' ', end='')
    print()
print()

print("Solution:")
for r in range(9):
    print('                      ', end='')
    for c in range(9):
        s = str(sol.get_var_solution(x[r][c]).get_value())
        if G[r][c]==0:
            s = '\x1b[1m'+s
        else:
            s = '\x1b[1;01m'+s            
        print(s+'\x1b[0m'+' ', end='')
    print()

Input grid:
                      8 . . . . . . . . 
                      . . 3 6 . . . . . 
                      . 7 . . 9 . 2 . . 
                      . 5 . . . 7 . . . 
                      . . . . 4 5 7 . . 
                      . . . 1 . . . 3 . 
                      . . 1 . . . . 6 8 
                      . . 8 5 . . . 1 . 
                      . 9 . . . . 4 . . 

Solution:
                      8 1 2 7 5 3 6 4 9 
                      9 4 3 6 8 2 1 7 5 
                      6 7 5 4 9 1 2 8 3 
                      1 5 4 2 3 7 8 9 6 
                      3 6 9 8 4 5 7 2 1 
                      2 8 7 1 6 9 5 3 4 
                      5 2 1 9 7 4 3 6 8 
                      4 3 8 5 2 6 9 1 7 
                      7 9 6 3 1 8 4 5 2 


# An optimization variant

In this variant, we suppose the grid has several feasible solution and we want to find a feasible solution that **minimizes the sum of the numbers in the two diagonals**. We use the same grid as before but remove some values.

In [5]:
RelaxedGrid = """
                . . . | . . . | . . . 
                . . 3 | 6 . . | . . .   
                . 7 . | . 9 . | 2 . .   
                ------+-------+------
                . 5 . | . . 7 | . . .   
                . . . | . . 5 | 7 . .   
                . . . | . . . | . 3 .   
                ------+-------+------
                . . 1 | . . . | . . .   
                . . 8 | 5 . . | . 1 .  
                . 9 . | . . . | 4 . .  
"""

EmptyGrid = """
                . . . | . . . | . . . 
                . . . | . . . | . . .   
                . . . | . . . | . . .   
                ------+-------+------
                . . . | . . . | . . .   
                . . . | . . . | . . .   
                . . . | . . . | . . .   
                ------+-------+------
                . . . | . . . | . . .   
                . . . | . . . | . . .  
                . . . | . . . | . . .  
"""

G = Read(RelaxedGrid)

In [6]:
# Create model object
model = CpoModel()

# Decision variables: x[r][c] is the value at row r column c
x = [ [integer_var(1,9) for c in range(9)] for r in range(9) ]

# Constraints: input grid values
model.add([x[r][c]==G[r][c] for r in range(9) for c in range(9) if G[r][c]!=0 ])            
            
# Constraints: different values in each row
model.add([all_diff([x[r][c] for c in range(9)]) for r in range(9)])

# Constraints: different values in each column    
model.add([all_diff([x[r][c] for r in range(9)]) for c in range(9)  ])

# Constraints: different values in each sub-square 
model.add([all_diff([x[r][c] for r in range(3*sr,3*sr+3) for c in range(3*sc,3*sc+3)]) for sr in range(3) for sc in range(3) ])
  
# Objective: minimize sum of the two diagonals
model.add(minimize(sum(x[r][r]+x[r][8-r] for r in range(9))))

In [7]:
# Solve the model
sol = model.solve(trace_log=True, LogPeriod=1000000)

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 81 variables, 43 constraints
 ! LogPeriod            = 1000000
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 155.5 (before), 155.5 (after)
 !  . Memory usage      : 336.1 kB (before), 336.1 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         81                 -
 + New bound is 36
                        0         65    1            -
 + New bound is 37
                        0         65    1   F        -
 + New bound is 38
 *            72       26  0.02s        1      (gap is 47.22%)
 *            68       20  0.02s        7      (gap is 44.12%)
 *            61      436  0.02s        7      (gap is 37.70%)
 *            51     1812  0.03s        1     

# Displaying the input grid and an optimal solution

In [8]:
print("Input grid:")
for r in range(9):
    print('                      ', end='')
    for c in range(9):
        if G[r][c]==0:
            s = '.'
            if c==r or c==8-r:
                s = '\x1b[1;43m'+s
            else:
                s = '\x1b[1m'+s
        else:
            s = str(G[r][c])
            if c==r or c==8-r:
                s = '\x1b[1;01;43m'+s
            else:
                s = '\x1b[1;01m'+s  
        print(s+'\x1b[0m'+' ', end='')
    print()
    
print()

print("Optimal solution:")
for r in range(9):
    print('                      ', end='')
    for c in range(9):
        s = str(sol.get_var_solution(x[r][c]).get_value())
        if G[r][c]==0:
            if c==r or c==8-r:
                s = '\x1b[1;43m'+s
            else:
                s = '\x1b[1m'+s
        else:
            if c==r or c==8-r:
                s = '\x1b[1;01;43m'+s
            else:
                s = '\x1b[1;01m'+s            
        print(s+'\x1b[0m'+' ', end='')
    print()
print()
print("Objective value = \x1b[1;43m" + str(sol.get_objective_values()[0])+'\x1b[0m')

Input grid:
                      . . . . . . . . . 
                      . . 3 6 . . . . . 
                      . 7 . . 9 . 2 . . 
                      . 5 . . . 7 . . . 
                      . . . . . 5 7 . . 
                      . . . . . . . 3 . 
                      . . 1 . . . . . . 
                      . . 8 5 . . . 1 . 
                      . 9 . . . . 4 . . 

Optimal solution:
                      2 4 6 7 3 8 5 9 1 
                      9 1 3 6 5 2 8 4 7 
                      8 7 5 1 9 4 2 6 3 
                      6 5 9 3 8 7 1 2 4 
                      1 3 4 9 2 5 7 8 6 
                      7 8 2 4 6 1 9 3 5 
                      5 6 1 2 4 9 3 7 8 
                      4 2 8 5 7 3 6 1 9 
                      3 9 7 8 1 6 4 5 2 

Objective value = 46


# Small grid example to illustrate the search tree

In [ ]:
SmallGrid = """
                1 . | 4 . 
                . . | . 2   
                ----+----
                . . | . . 
                . . | . . 
 
"""
G = Read(SmallGrid,size=4)

# Create model object
model = CpoModel()

# Decision variables: x[r][c] is the value at row r column c
x = [ [integer_var(1,4, name='x'+str(r+1)+str(c+1)) for c in range(4)] for r in range(4) ]

# Constraints: input grid values
model.add([x[r][c]==G[r][c] for r in range(4) for c in range(4) if G[r][c]!=0 ])            
            
# Constraints: different values in each row
model.add([all_diff([x[r][c] for c in range(4)]) for r in range(4)])

# Constraints: different values in each column    
model.add([all_diff([x[r][c] for r in range(4)]) for c in range(4)  ])

# Constraints: different values in each sub-square 
model.add([all_diff([x[r][c] for r in range(2*sr,2*sr+2) for c in range(2*sc,2*sc+2)]) for sr in range(2) for sc in range(2) ])
  
# Objective: minimize sum of the two diagonals
model.add(minimize(sum(x[r][r]+x[r][3-r] for r in range(4))))

# Solve the model
sol = model.solve(trace_log=True, LogPeriod=1, Workers=1, SearchType='DepthFirst')

print("Input grid:")
for r in range(4):
    print('                      ', end='')
    for c in range(4):
        if G[r][c]==0:
            s = '.'
            if c==r or c==3-r:
                s = '\x1b[1;43m'+s
            else:
                s = '\x1b[1m'+s
        else:
            s = str(G[r][c])
            if c==r or c==3-r:
                s = '\x1b[1;01;43m'+s
            else:
                s = '\x1b[1;01m'+s  
        print(s+'\x1b[0m'+' ', end='')
    print()
    
print()

print("Optimal solution:")
for r in range(4):
    print('                      ', end='')
    for c in range(4):
        s = str(sol.get_var_solution(x[r][c]).get_value())
        if G[r][c]==0:
            if c==r or c==3-r:
                s = '\x1b[1;43m'+s
            else:
                s = '\x1b[1m'+s
        else:
            if c==r or c==3-r:
                s = '\x1b[1;01;43m'+s
            else:
                s = '\x1b[1;01m'+s            
        print(s+'\x1b[0m'+' ', end='')
    print()
print()
print("Objective value = \x1b[1;43m" + str(sol.get_objective_values()[0])+'\x1b[0m')